# Intro of this file
+ Performs 5 fold cv (main args might want to change: batchsize=8, fold=5, ep=10)
+ To change the fold numbers, simply change "CVfolds" in the section "Start cross validation"
+ loader and train is modified compared to the main branch


In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
import numpy as np
import random
import matplotlib.pyplot as plt

from utils.loader import DataLoader
from models.effnet_encoder import EffnetEncoder
from models.mtl_framework import MTLFramework
from utils import tools, config

In [2]:
# Set configs
batch_size = 8
batch_size_val = 8
num_train, num_val, num_test = config.config['num_train'], config.config['num_val'], config.config['num_test']
img_height, img_width, channels = config.config['input_shape']

### Build our MTL

In [3]:
def build_model(img_height, img_width, channels):
    ### CLEARS OLD MODELS IN CACHE
    tf.keras.backend.clear_session()

    # Get encoder
    base_model_name = 'B0'
    encoder = EffnetEncoder(base_model_name, (img_height, img_width, channels)).build_encoder(trainable=True)

    # Use our MTL framework to custom build a model
    mtl_builder = MTLFramework(encoder, (img_height, img_width, channels))
    mtl_builder.add_segmentation_head()
    mtl_builder.add_binary_classification_head(base_model_name, trainable=True)
    mtl_builder.add_bbox_classification_head(base_model_name, trainable=True)
    model = mtl_builder.build_mtl_model()

    return model

### Train model

In [4]:
def generator_img(img_ds, masks_ds, label_ds, bbox_ds):
    ''' Merges together datasets into a unified generator to pass for training '''
    a = img_ds.as_numpy_iterator()
    b = masks_ds.as_numpy_iterator()
    c = label_ds.as_numpy_iterator()
    d = bbox_ds.as_numpy_iterator()
    
    while True:
        X = a.next()
        Y1 = b.next()
        Y2 = c.next()
        Y3 = d.next()
        
        # Regularisation and shuffling
        X, Y1, Y2, Y3 = tools.get_randomised_data([X, Y1, Y2, Y3])
        # X, Y1, Y3 = tools.data_augmentation(X, Y1, Y3) # Fix augmentation
        
        yield X, (Y1, Y2, Y3)

In [5]:
def generator_img_val(img_ds_val, masks_ds_val, label_ds_val, bbox_ds):
    ''' Merges together datasets into a unified generator to pass for training '''
    a = img_ds_val.as_numpy_iterator()
    b = masks_ds_val.as_numpy_iterator()
    c = label_ds_val.as_numpy_iterator()
    d = bbox_ds.as_numpy_iterator()
    
    while True:
        X = a.next()
        Y1 = b.next()
        Y2 = c.next()
        Y3 = d.next()
        
        yield X, (Y1, Y2, Y3)

### Start cross validation

In [6]:
##### Change this to modify the folds #####
CVfolds = 5
###########################################
val_acc = []
num_train_cv = num_train * (CVfolds-1)/CVfolds
num_val_cv = num_train * 1/CVfolds

for cv_iter in range(CVfolds):
      # CrossVal: 0 = no CV, 1 = training set, 2 = val set
      loader1 = DataLoader(batch_size=batch_size, batch_size_val=batch_size_val, CrossVal=1, CV_iteration=cv_iter, fold=CVfolds)
      loader2 = DataLoader(batch_size=batch_size, batch_size_val=batch_size_val, CrossVal=2, CV_iteration=cv_iter, fold=CVfolds)

      # Train set
      img_ds = loader1.get_image_ds().repeat()
      masks_ds = loader1.get_mask_ds().repeat()
      label_ds = loader1.get_binary_ds().repeat()
      bbox_ds = loader1.get_bboxes_ds().repeat()

      # Validation set
      img_ds_val = loader2.get_image_ds().repeat()
      masks_ds_val = loader2.get_mask_ds().repeat()
      label_ds_val = loader2.get_binary_ds().repeat()
      bbox_ds_val = loader2.get_bboxes_ds().repeat()
      
      model = build_model(img_height, img_width, channels)

      model.compile(optimizer=keras.optimizers.Adam(1e-4),
                  loss={'segnet_out' : tf.keras.losses.BinaryCrossentropy(from_logits=True),
                        'bin_class_out' : tf.keras.losses.BinaryCrossentropy(),
                        'bbox_out' : tf.keras.losses.MeanAbsoluteError()},
                  loss_weights=[1,1,1/100], # Scale MAE to BC range
                  metrics=['accuracy'])

      print(f"-------------------- start cross val {cv_iter+1}/{CVfolds} --------------------")

      history = model.fit(generator_img(img_ds, masks_ds, label_ds, bbox_ds), 
                        validation_data=generator_img_val(img_ds_val, masks_ds_val, label_ds_val, bbox_ds), 
                        epochs=10, 
                        steps_per_epoch=num_train_cv//batch_size, 
                        validation_steps=num_val_cv//batch_size_val)

      # save the final val_acc of each cv step
      val_acc.append([history.history['val_segnet_out_accuracy'][-1], 
                              history.history['val_bin_class_out_accuracy'][-1],
                              history.history['val_bbox_out_accuracy'][-1]])

      # print("\n")

-------------------- start cross val 1/5 --------------------
Epoch 1/10
221/221 [==============================] - 88s 342ms/step - loss: 1.7879 - segnet_out_loss: 0.4527 - bin_class_out_loss: 0.2282 - bbox_out_loss: 110.6995 - segnet_out_accuracy: 0.7834 - bin_class_out_accuracy: 0.9202 - bbox_out_accuracy: 0.2675 - val_loss: 1.4185 - val_segnet_out_loss: 0.2564 - val_bin_class_out_loss: 0.0409 - val_bbox_out_loss: 112.1228 - val_segnet_out_accuracy: 0.9109 - val_bin_class_out_accuracy: 0.9886 - val_bbox_out_accuracy: 0.3773
Epoch 2/10
221/221 [==============================] - 74s 336ms/step - loss: 1.3661 - segnet_out_loss: 0.2284 - bin_class_out_loss: 0.0527 - bbox_out_loss: 108.4978 - segnet_out_accuracy: 0.9188 - bin_class_out_accuracy: 0.9870 - bbox_out_accuracy: 0.4084 - val_loss: 1.2624 - val_segnet_out_loss: 0.1772 - val_bin_class_out_loss: 0.0277 - val_bbox_out_loss: 105.7455 - val_segnet_out_accuracy: 0.9375 - val_bin_class_out_accuracy: 0.9909 - val_bbox_out_accuracy: 0.4

In [14]:
# Print out the results to see offline
print("\tsegnet \tbinary \tbbox")
for i in range(CVfolds):
    print(f"CV {i+1} \t{np.round(val_acc[i][0], 4)} \t{np.round(val_acc[i][1], 4)} \t{np.round(val_acc[i][2], 4)}")

print("-----------------------------")
avg = np.round(np.mean(val_acc, axis=0), 4)
std = np.round(np.std(val_acc, axis=0), 4)
print(f"Avg  \t{avg[0]} \t{avg[1]} \t{avg[2]}")
print(f"Std  \t{std[0]} \t{std[1]} \t{std[2]}")


	segnet 	binary 	bbox
CV 1 	0.9527 	0.9886 	0.6864
CV 2 	0.9561 	0.9886 	0.6705
CV 3 	0.9522 	0.9955 	0.6977
CV 4 	0.9557 	0.9909 	0.6273
CV 5 	0.952 	0.9977 	0.6409
-----------------------------
Avg  	0.9537 	0.9923 	0.6645
Std  	0.0018 	0.0037 	0.0267


In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
#ax.plot(list(range(10)), history.history['segnet_out_accuracy'], 'r-', label='Segmentation - Training Accuracy')
ax.plot(list(range(10)), history.history['val_segnet_out_accuracy'], 'r--', label='Segmentation - Validation Accuracy')
#ax.plot(list(range(10)), history.history['bin_class_out_accuracy'], 'c-', label='Classification - Training Accuracy')
ax.plot(list(range(10)), history.history['val_bin_class_out_accuracy'], 'c--', label='Classification - Validation Accuracy')
ax2 = ax.twinx()
#ax.plot(list(range(10)), history.history['bbox_out_accuracy'], 'm-', label='Bounding Box - Training Accuracy')
ax.plot(list(range(10)), history.history['val_bbox_out_accuracy'], 'm--', label='Bounding Box - Validation Accuracy')
ax.legend()
ax.set_xlabel('Epochs')
ax.set_ylabel('Segmentation/Classification Accuracy')
ax2.set_ylabel('Bounding Box Accuracy')

In [ ]:
model.save('EffishingNetN')

## Test on test-set

In [39]:
model = tf.keras.models.load_model('EffishingNetN')

In [6]:
# Load test-set
img_ds_test = loader.get_image_ds(test_mode=True)
masks_ds_test = loader.get_mask_ds(test_mode=True)
label_ds_test = loader.get_binary_ds(test_mode=True)
bbox_ds_test = loader.get_bboxes_ds(test_mode=True)

In [ ]:
# Predict on test-set
seg_pred, bin_pred, bbox_pred = model.predict(img_ds_test, batch_size=10)
seg_pred = tf.where(seg_pred >= 0, 1, 0) # Convert to {0,1} binary classes
bin_pred = np.round(bin_pred) # Round confidence score

bin_acc = np.sum(bin_pred == label_ds_test)/label_ds_test.shape[0]
seg_acc = np.sum(seg_pred == masks_ds_test)/(masks_ds_test.shape[0]*(img_height*img_width))
iou = np.mean(tools.calculate_iou(bbox_ds_test, bbox_pred))
print(f'Binary Acc: {round(bin_acc*100, 3)}%,   Seg Acc: {round(seg_acc*100, 3)}%,    BBox IOU: {round(iou*100, 3)}%')

In [58]:
def show_seg_pred(img: np.array, mask_truth: np.array, mask_pred: np.array, bbox_truth: np.array, bbox_pred: np.array):
    ''' Show segmentation prediction with bounding box pred '''
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(12,12))
    seg_max = tf.where(mask_pred > 0, 1, 0)
    box_img_truth = tf.image.draw_bounding_boxes(tf.cast(tf.expand_dims(img, 0), tf.float32), tools.fix_bbox(bbox_truth).reshape([1,1,4])/256, np.array([[255, 0, 0]]))
    box_img = tf.image.draw_bounding_boxes(tf.cast(tf.expand_dims(img, 0), tf.float32), tools.fix_bbox(bbox_pred).reshape([1,1,4])/256, np.array([[0, 255, 0]]))
    
    ax1.imshow(tf.keras.utils.array_to_img(tf.squeeze(box_img_truth)))
    ax2.imshow(tf.keras.utils.array_to_img(tf.squeeze(box_img)))
    ax3.imshow(tf.keras.utils.array_to_img(mask_truth))
    ax4.imshow(tf.keras.utils.array_to_img(seg_max[0]))
    ax1.set_title('Truth')
    ax2.set_title('Prediction')
    ax3.set_title('Truth')
    ax4.set_title('Prediction')

In [ ]:
# Visualise predictions
idx = list(range(img_ds_test.shape[0]))
random.shuffle(idx)
for i in range(3):
    show_seg_pred(img_ds_test[idx[i]], masks_ds_test[idx[i]], seg_pred[idx[i]][tf.newaxis, ...], bbox_ds_test[idx[i]], bbox_pred[idx[i]])